In [2]:
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [3]:
trainData = pd.read_csv('Glass_Quality_Participants_Data/Train.csv')

In [ ]:
# Number of columns = 16  Train
# Number of rows = 1358   Train

# Number of columns = 15  Test
# Number of rows = 583   Test

In [ ]:
# Columns
# Index(['grade_A_Component_1', 'grade_A_Component_2', 'max_luminosity',
#        'thickness', 'xmin', 'xmax', 'ymin', 'ymax', 'pixel_area', 'log_area',
#        'x_component_1', 'x_component_2', 'x_component_3', 'x_component_4',
#        'x_component_5', 'class'],
#       dtype='object')

In [4]:
features = trainData.iloc[:,:-1]
labels = trainData.iloc[:,-1]

In [5]:
labels = labels.values.reshape(-1,1)
# labels = np.array([i[0]-1 for i in labels]).reshape(-1,1)
labels.shape

(1358, 1)

In [6]:
def ohencoder(labels):
    return np.array([[0,1] if i[0] == 2 else [1,0] for i in labels])

ohencoded_labels = ohencoder(labels)

In [ ]:
ohencoded_labels.shape

In [7]:
features = features.astype('float')

In [ ]:
features.info()

In [ ]:
# x and grade_A are categorical variables
# No missing values

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

features.hist(column=["xmax", "xmin","ymin","ymax",])

In [ ]:
features.hist(column=["max_luminosity","pixel_area","log_area"])#,,,

In [ ]:
features.corr()

In [ ]:
# pixel_area and log_area
# xmin and xmax
# ymin and ymax
# The above pairs have a very high degree of positive correlation

In [8]:
features = features.drop(columns=['pixel_area', 'xmax','ymax'])

In [9]:
tfeatures = features.loc[:,['max_luminosity','thickness','xmin','ymin','log_area']]

In [10]:
scaler = StandardScaler()
scaler.fit(tfeatures)
tfeatures = scaler.transform(tfeatures)

In [11]:
tfeatures = pd.DataFrame(tfeatures,columns=['max_luminosity','thickness','xmin','ymin','log_area'])

In [12]:
features = features.drop(columns=['max_luminosity','thickness','xmin','ymin','log_area'])

In [13]:
normalized_features = pd.concat([features,tfeatures],axis=1)

In [ ]:
normalized_features.shape

In [ ]:
normalized_features.corr()

In [14]:
np_features = normalized_features.to_numpy()
np_labels = ohencoded_labels

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np_features, np_labels, test_size=0.33, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [16]:
from keras import models, layers, optimizers
model = models.Sequential()
model.add(layers.Dense(10, activation = 'relu', input_dim = 12))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation = 'relu'))
model.add(layers.Dense(8, activation = 'relu'))
model.add(layers.Dense(2, activation = 'softmax'))


model.compile(optimizer = optimizers.adam(lr = 1e-3),
              loss = 'categorical_crossentropy',
              metrics = [tf.keras.metrics.CategoricalAccuracy()]
             )

In [17]:
my_callbacks = [
    EarlyStopping(patience=5),
    
    
]

In [18]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, callbacks=my_callbacks, validation_data=[X_test, y_test]
)


Train on 909 samples, validate on 449 samples
Epoch 1/100
909/909 [==============================] - 0s 192us/step - loss: 0.6725 - categorical_accuracy: 0.6639 - val_loss: 0.6798 - val_categorical_accuracy: 0.6561
Epoch 2/100
909/909 [==============================] - 0s 56us/step - loss: 0.6590 - categorical_accuracy: 0.6555 - val_loss: 0.6615 - val_categorical_accuracy: 0.6545
Epoch 3/100
909/909 [==============================] - 0s 51us/step - loss: 0.6421 - categorical_accuracy: 0.6533 - val_loss: 0.6454 - val_categorical_accuracy: 0.6540
Epoch 4/100
909/909 [==============================] - 0s 50us/step - loss: 0.6276 - categorical_accuracy: 0.6539 - val_loss: 0.6259 - val_categorical_accuracy: 0.6538
Epoch 5/100
909/909 [==============================] - 0s 51us/step - loss: 0.6201 - categorical_accuracy: 0.6521 - val_loss: 0.6103 - val_categorical_accuracy: 0.6537
Epoch 6/100
909/909 [==============================] - 0s 48us/step - loss: 0.6019 - categorical_accuracy: 0.652

Epoch 48/100
909/909 [==============================] - 0s 48us/step - loss: 0.3639 - categorical_accuracy: 0.7529 - val_loss: 0.3100 - val_categorical_accuracy: 0.7537
Epoch 49/100
909/909 [==============================] - 0s 44us/step - loss: 0.3536 - categorical_accuracy: 0.7545 - val_loss: 0.3087 - val_categorical_accuracy: 0.7553
Epoch 50/100
909/909 [==============================] - 0s 47us/step - loss: 0.3591 - categorical_accuracy: 0.7561 - val_loss: 0.3078 - val_categorical_accuracy: 0.7568
Epoch 51/100
909/909 [==============================] - 0s 50us/step - loss: 0.3715 - categorical_accuracy: 0.7574 - val_loss: 0.3096 - val_categorical_accuracy: 0.7580
Epoch 52/100
909/909 [==============================] - 0s 48us/step - loss: 0.3555 - categorical_accuracy: 0.7585 - val_loss: 0.3079 - val_categorical_accuracy: 0.7592
Epoch 53/100
909/909 [==============================] - 0s 46us/step - loss: 0.3601 - categorical_accuracy: 0.7599 - val_loss: 0.3059 - val_categorical_acc

In [ ]:
ypreds = model.predict(X_test)

In [ ]:
ypreds

In [ ]:
samplesubmission = pd.DataFrame(ypreds, columns = ['1','2'])

In [ ]:
samplesubmission

In [ ]:
def ontocat(arr):
    return np.array([0 if i[0]==1 else 1 for i in arr])

catypreds = ontocat(ypreds)
catytest = ontocat(y_test)

# Validation

In [19]:
testData = pd.read_csv('Glass_Quality_Participants_Data/Test.csv')

In [20]:
testfeatures = testData.drop(columns=['pixel_area', 'xmax','ymax'])

In [21]:
testfeatures = testfeatures.astype('float')

In [22]:
tfeatures = testfeatures.loc[:,['max_luminosity','thickness','xmin','ymin','log_area']]

scaler = StandardScaler()
scaler.fit(tfeatures)
tfeatures = scaler.transform(tfeatures)

tfeatures = pd.DataFrame(tfeatures,columns=['max_luminosity','thickness','xmin','ymin','log_area'])

testfeatures = testfeatures.drop(columns=['max_luminosity','thickness','xmin','ymin','log_area'])

normalized_features = pd.concat([testfeatures,tfeatures],axis=1)

In [23]:
np_testfeatures = normalized_features.to_numpy()

In [24]:
testpreds = model.predict(np_testfeatures)

In [25]:
testpreds_df = pd.DataFrame(testpreds, columns=['1','2'])

In [26]:
testpreds_df.to_excel('submission4.xlsx', index=False)